# 001-maps

This notebook loads all the .fit files in a directory, reads their records messages into pandas DataFrames, and plots the GPS coordinates on a map.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fitparse import FitFile
from tqdm.notebook import tqdm
from fitanalyser import fitparser
import folium

# Load fit files

In [ ]:
file_dir = os.path.join('..', 'data', 'fit')

In [ ]:
file_paths = [os.path.join(file_dir, file_name) for file_name in os.listdir(file_dir)]

In [ ]:
list_of_records = []
for file_path in tqdm(file_paths):
    fitfile = FitFile(file_path)
    records = fitparser.get_records(fitfile)
    records = records.dropna(subset=[('position_long', 'semicircles'), ('position_lat', 'semicircles')])
    records.loc[:, ('position_long', 'deg')] = records['position_long']['semicircles']*180/2**31
    records.loc[:, ('position_lat', 'deg')] = records['position_lat']['semicircles']*180/2**31
    list_of_records.append(records)

print(list_of_records[ 0].index[ 0])
print(list_of_records[-1].index[-1])

# Correct the timestamps for a different timezone

This should only be done once because modifying the `records` in the `for` loop modifies the `list_of_records` element permanently.

In [ ]:
timezone = pd.Timedelta(hours=-4)

In [ ]:
print(list_of_records[0].index[0])
for records in list_of_records:
    records.index = records.index + timezone
print(list_of_records[0].index[0])

# Plot the GPS coordinates of all records

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[16, 20])
for records in list_of_records:
    ax.plot(
        records['position_long']['deg'],
        records['position_lat']['deg'],
        color='k',
    )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('equal')

# Save figure as vector eps file
fig.savefig(os.path.join('..','figures','map.eps'), format='eps')

In [ ]:
# Calculate the map centre from the average coordinates of all records
records_centres = [
    [records['position_lat']['deg'].mean(), records['position_long']['deg'].mean()] 
    for records in list_of_records]
map_centre = np.array(records_centres).mean(axis=0)

# Build map 
folium_map = folium.Map(
    location=map_centre, 
    zoom_start=11, 
    tiles='OpenStreetMap',
    width=640, 
    height=800,
)

# Add GPS coordinates to map
for records in list_of_records:
    folium.ColorLine(
        records.loc[:, [('position_lat', 'deg'), ('position_long', 'deg')]].values,
        colors=range(len(records)),
        colormap=['black', 'black'],
        fill_color='#0080bb',
    ).add_to(folium_map)

# Save map to html file
folium_map.save(os.path.join('..','figures','map.html'))

# Display map in Jupyter
folium_map

# Plot the GPS coordinates of a given date

In [ ]:
date = '2019-02-21'

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[16, 20])
for records in list_of_records:
    ax.plot(
        records['position_long']['deg'],
        records['position_lat']['deg'],
        color='k',
    )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('equal')

for records in list_of_records:
    records = records.loc[date:date].copy()
    ax.plot(
        records['position_long']['deg'],
        records['position_lat']['deg'],
        color='r',
        linewidth=3,
    )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('equal')

# Save figure as vector eps file
fig.savefig(os.path.join('..','figures','map.eps'), format='eps')